# ADVANCED PSE+ STAKEHOLDER SUMMIT I 2023: 
# Interactive Code Demonstration Using WaterTAP

### Today's demonstration will show how to build, initialize, simulate, and optimize a flowsheet for multiperiod analysis.

# Overall approach to multiperiod flowsheets
<p align="center">
  <img src="assets/MP_Framework.png" width="800">
</p>

## Part 1: Build, setup, and simulate the multiperiod RO+PV+Battery flowsheet


<p align="center">
  <img src="assets/RO_PV_Batt.svg" width="1000">
</p>

## Multiperiod Setup

### Quick and high-level overview of setting up steady-state and surrogate parts
#### Import Pyomo, IDAES, and WaterTAP packages

In [ ]:
# Pyomo imports
from pyomo.environ import ConcreteModel, Objective, Var, value, units as pyunits

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.apps.grid_integration.multiperiod.multiperiod import MultiPeriodModel
from idaes.core.solvers.get_solver import get_solver


#### Build Model

In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

#### Import and define the Reverse Osmosis unit model

In [ ]:
from steady_state_flowsheets.simple_RO_unit import ROUnit
m.fs.RO = ROUnit()

#### This simple RO unit will assume a steady-state system with constant production and power demand
        ROUnit.product = 6000 # m3/day
        ROUnit.power_demand = 1 # MW

#### Import and define the Battery model

In [ ]:
from steady_state_flowsheets.battery import BatteryStorage 
# From DISPATCHES 
m.fs.battery = BatteryStorage()

<p align="center">
  <img src="assets/var_table.png" width="580">
</p>

#### Load PV surrogate

In [ ]:
from idaes.core.surrogate.pysmo_surrogate import PysmoSurrogate
PV_surrogate = PysmoSurrogate.load_from_file('assets/demo_surrogate.json')

<p align="center">
  <img src="assets/solar_cycle.png" width="380">
</p>

##### The PV surrogate will predict the energy production of a given PV system based on:
* PV system design size [kW] (peak power )
* Day of the year
* Hour of the Day

### Define critical variables and add energy balance constraints in one time step/ steady-state conditions

In [ ]:
from steady_state_flowsheets.system import *
define_system_vars(m)
add_steady_state_constraints(m)

#### System-level variables include:
| Var                       | Description                                    | Unit |
| :----:                    | :----:                                    |:----:|
| pv_to_ro           | Energy supplied to the RO coming from PV | kW  |
| grid_to_ro          | Energy supplied to the RO coming from grid | kW |
| curtailment   | PV curtailment                 | kW |
| elec_price                   | Electricity Price                                 | $/kWh  |
| pv_gen                   | Energy in                                 | kWh  |

### The steady state constraints:

<p align="center">
  <img src="assets/Balance.png" width="700">
</p>
<p align="center">
  <img src="assets/constraint.png" width="700">
</p>

In [ ]:
m.fs.battery.nameplate_energy.fix(8000) # Battery capacity [kWh]
m.fs.battery.nameplate_power.fix(400) # Battery power [kW]
m.fs.pv_gen.fix(700) # PV generation [kW]

In [ ]:
solver = get_solver()

m.fs.battery.initialize()
results = solver.solve(m)

In [ ]:
print_system_results(m)

## Refresh - approach to multiperiod flowsheets
<p align="center">
  <img src="assets/MP_Framework_2.png" height="380">
</p>

In [ ]:
def get_pv_ro_variable_pairs(t1, t2):
    """
    This function returns pairs of variables that need to be connected across two time periods

    Args:
        t1: current time block
        t2: next time block

    Returns:
        None
    """
    return [
        (t1.fs.battery.state_of_charge[0], t2.fs.battery.initial_state_of_charge),
        (t1.fs.battery.energy_throughput[0], t2.fs.battery.initial_energy_throughput),
        (t1.fs.battery.nameplate_power, t2.fs.battery.nameplate_power),
        (t1.fs.battery.nameplate_energy, t2.fs.battery.nameplate_energy),
        ]

<p align="center">
  <img src="assets/linking.png" width="520">
</p>

#### Define the flowsheet options

In [ ]:
def create_multiperiod_pv_battery_model(
        n_time_points= 24,
        ro_capacity = 6000, # m3/day
        ro_elec_req = 1000, # kW
        pv_oversize = 1,
        surrogate = None):
    
    # create the multiperiod object
    '''The `MultiPeriodModel` class helps transfer existing steady-state
    process models to multiperiod versions that contain dynamic time coupling'''
    mp = MultiPeriodModel(
        n_time_points=n_time_points,
        process_model_func= steady_state_flowsheet,
        linking_variable_func= get_pv_ro_variable_pairs,
        initialization_func= m.fs.battery.initialize(),
        unfix_dof_func= unfix_dof,
    )

    # process_model_func - This is our steady-state or single time instance flowsheet
    # unfix_dof - This is where we fix or unfix battery size, power, etc...
    # initialize_system - This is where we initialize the battery
    
    # Flowsheet options is where we define the input options for the steady-state flowsheet
    flowsheet_options={ t: { 
                            "pv_gen": eval_surrogate(surrogate, design_size = 1000, Day = 1, Hour = t%24),
                            "electricity_price": get_elec_tier(Hour = t%24),
                            "ro_capacity": ro_capacity, 
                            "ro_elec_req": ro_elec_req,
                            "pv_oversize": pv_oversize} 
                            for t in range(n_time_points)
    }

    # Build a multi-period capable model using user-provided functions
    mp.build_multi_period_model(
        model_data_kwargs=flowsheet_options)
    
    # Fix the initial state of charge to zero
    mp.blocks[0].process.fs.battery.initial_state_of_charge.fix(0)

    # Set capital costs and the objective function that apply across all time periods
    add_pv_ro_constraints(mp)

    # Total Costs = Battery Costs + PV Costs + RO Costs
    # LCOW = Total Costs / RO Production [$/m3]

    return mp

<p align="center">
  <img src="assets/Multiperiod.png" width="900">
</p>

# Simulation

In [ ]:
mp = create_multiperiod_pv_battery_model(surrogate=PV_surrogate)

In [ ]:
results = solver.solve(mp)
print_results(mp)

In [ ]:
print([value(mp.blocks[i].process.fs.battery.state_of_charge[0]) for i in range(24)])

In [ ]:
from util.visualize import create_plot
create_plot(mp)

## Now let's optimize the battery

#### Unfix 
* Battery Nameplate Power
* Battery Nameplate Energy

In [ ]:
mp_optimized = optimize_multiperiod_pv_battery_model(surrogate=PV_surrogate)
results = solver.solve(mp_optimized)
print_results(mp_optimized)
create_plot(mp_optimized)